In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\folks\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Train Data Preprocessing

In [2]:
import csv
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import string
table = str.maketrans('', '', string.punctuation)
sentences = []
labels = []
stopwords = stopwords.words("english")

with open('train.csv', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=",")
    for row in reader:
        labels.append(int(row[4]))
        sentence = row[3].lower()
        sentence = sentence.replace("," , " , ")
        sentence = sentence.replace("." , " . ")
        sentence = sentence.replace("!" , " ! ")
        sentence = sentence.replace("?" , " ? ")
        sentence = sentence.replace("." , " . ")
        soup = BeautifulSoup(sentence)
        sentence = soup.get_text()
        words = sentence.split()
        filtered_text = ""
        for word in words:
            word = word.translate(table)
            if word not in stopwords:
                filtered_text = filtered_text + word + " "
        sentences.append(filtered_text)

C:\Users\folks\AppData\Local\Temp\ipykernel_17220\2640853100.py:20: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(sentence)


check whether the train data is loaded.

In [3]:
print(sentences[0])

deeds reason earthquake may allah forgive us 


## Test Data Preprocessing

In [4]:
test_sentences = []

with open('test.csv', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=",")
    for row in reader:
        sentence = row[3].lower()
        sentence = sentence.replace("," , " , ")
        sentence = sentence.replace("." , " . ")
        sentence = sentence.replace("!" , " ! ")
        sentence = sentence.replace("?" , " ? ")
        sentence = sentence.replace("." , " . ")
        soup = BeautifulSoup(sentence)
        sentence = soup.get_text()
        words = sentence.split()
        filtered_text = ""
        for word in words:
            word = word.translate(table)
            if word not in stopwords:
                filtered_text = filtered_text + word + " "
        test_sentences.append(filtered_text)

C:\Users\folks\AppData\Local\Temp\ipykernel_17220\3574024443.py:12: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(sentence)


Check whether the data is loaded.

In [5]:
print(test_sentences[0])

happened terrible car crash 


### Model Phase

In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocal_size = 25000
max_length = 10


tokenizer = Tokenizer(num_words=vocal_size, oov_token='<OOV>')
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(sentences)

training_padded = pad_sequences(training_sequences, maxlen=max_length, padding='post', truncating='post')

testing_sequences = tokenizer.texts_to_sequences(test_sentences)

testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding='post', truncating='post')

In [7]:
import numpy as np

training_padded = np.array(training_padded)
labels = np.array(labels)
testing_padded = np.array(testing_padded)

In [8]:
print(word_index)

{'<OOV>': 1, 'httpt': 2, 'httpst': 3, 'like': 4, 'im': 5, 'fire': 6, 'get': 7, 'new': 8, 'via': 9, 'people': 10, 'news': 11, 'one': 12, 'dont': 13, '2': 14, 'video': 15, 'emergency': 16, 'disaster': 17, 'police': 18, 'would': 19, 'still': 20, 'u': 21, 'us': 22, 'body': 23, 'burning': 24, 'crash': 25, 'back': 26, 'storm': 27, 'california': 28, 'suicide': 29, 'got': 30, 'time': 31, '3': 32, 'know': 33, 'man': 34, 'buildings': 35, 'day': 36, 'rt': 37, 'first': 38, 'see': 39, 'world': 40, 'going': 41, 'bomb': 42, 'cant': 43, 'fires': 44, 'attack': 45, 'nuclear': 46, 'love': 47, 'youtube': 48, 'two': 49, 'dead': 50, 'killed': 51, 'go': 52, 'train': 53, '1': 54, 'full': 55, 'car': 56, 'war': 57, 'accident': 58, 'today': 59, 'hiroshima': 60, 'may': 61, 'families': 62, 'life': 63, 'good': 64, 'think': 65, 'say': 66, 'watch': 67, 'many': 68, 'last': 69, '4': 70, 'could': 71, 'want': 72, 'way': 73, 'years': 74, 'home': 75, 'make': 76, '5': 77, 'collapse': 78, 'work': 79, 'look': 80, 'please': 81

In [9]:
print(training_sequences[0])
print(training_padded[0])

[4408, 723, 155, 61, 1462, 4409, 22]
[4408  723  155   61 1462 4409   22    0    0    0]


### Train Model (From Lecture 6)

In [10]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocal_size, 16),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32 , dropout=0.2 return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, dropout=0.2)),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [11]:
adam = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)

model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

NameError: name 'model' is not defined